In [3]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
from datetime import date
import matplotlib.pyplot as plt

from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np 
import re
import seaborn as sns
from shapely.geometry import Point, Polygon
from shapely.geometry import MultiPolygon
import folium

import plotly.graph_objs as go  
import plotly.express as px

In [4]:
data = pd.read_csv('covid_impact_on_airport_traffic.csv')
data = data.drop(index=0)
data.head()

,AggregationMethod,Date,Version,AirportName,PercentOfBaseline,Centroid,City,State,ISO_3166_2,Country,Geography
1,Daily,2020-04-13,1.0,Kingsford Smith,29,POINT(151.180087713813 -33.9459774986125),Sydney,New South Wales,AU,Australia,"POLYGON((151.164354085922 -33.9301772341877, 1..."
2,Daily,2020-07-10,1.0,Kingsford Smith,54,POINT(151.180087713813 -33.9459774986125),Sydney,New South Wales,AU,Australia,"POLYGON((151.164354085922 -33.9301772341877, 1..."
3,Daily,2020-09-02,1.0,Kingsford Smith,18,POINT(151.180087713813 -33.9459774986125),Sydney,New South Wales,AU,Australia,"POLYGON((151.164354085922 -33.9301772341877, 1..."
4,Daily,2020-10-31,1.0,Kingsford Smith,22,POINT(151.180087713813 -33.9459774986125),Sydney,New South Wales,AU,Australia,"POLYGON((151.164354085922 -33.9301772341877, 1..."
5,Daily,2020-04-14,1.0,Kingsford Smith,59,POINT(151.180087713813 -33.9459774986125),Sydney,New South Wales,AU,Australia,"POLYGON((151.164354085922 -33.9301772341877, 1..."


In [5]:
data["Date"] = data["Date"].map(lambda x: date.fromisoformat(x))

In [6]:
data["weekday"] = data["Date"].map(lambda x: x.weekday())
w_list = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
data["weekday"] = [w_list[idx] for idx in data["weekday"]]

In [7]:
def cut_long(point):
    long, _ = point[6:-1].split(" ")
    return float(long)

def cut_lat(point):
    _, lat = point[6:-1].split(" ")
    return float(lat)

In [8]:
data["long"] = data["Centroid"].map(cut_long)
data["lat"] = data["Centroid"].map(cut_lat)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7246 entries, 1 to 7246
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   AggregationMethod  7246 non-null   object 
 1   Date               7246 non-null   object 
 2   Version            7246 non-null   float64
 3   AirportName        7246 non-null   object 
 4   PercentOfBaseline  7246 non-null   int64  
 5   Centroid           7246 non-null   object 
 6   City               7246 non-null   object 
 7   State              7246 non-null   object 
 8   ISO_3166_2         7246 non-null   object 
 9   Country            7246 non-null   object 
 10  Geography          7246 non-null   object 
 11  weekday            7246 non-null   object 
 12  long               7246 non-null   float64
 13  lat                7246 non-null   float64
dtypes: float64(3), int64(1), object(10)
memory usage: 849.1+ KB


In [10]:
df_Country_count = pd.DataFrame(data["Country"].value_counts())

In [11]:
df_Country_count["Country"]

United States of America (the)    4441
Canada                            2311
Australia                          256
Chile                              238
Name: Country, dtype: int64

In [12]:
data_aux = data[~data[["AirportName"]].duplicated()].reset_index(drop=True)

In [13]:
fig3 = px.bar(data_aux, x = 'Country', y = "AirportName",color="Country",title="Aereopuertos por pais" )

In [14]:
fig2 = px.bar(data_aux, x = 'AirportName', y = "AirportName",color="AirportName",title="Aereopuertos distribuidos" )

In [15]:
df_month_count = pd.DataFrame(data["Date"].map(lambda d: d.month).value_counts())
df_month_count = df_month_count.reset_index()
df_month_count = df_month_count.rename(columns={"Date":"Cantidad", "index":"mes"})
fig4 = px.bar(df_month_count, y="Cantidad", x= "mes", color="mes", title="Candtidad de movimiento durante la pandemia por mes")

In [19]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.H1(children='Trafico Aereo Durante el Covid-19'),

    html.Div(children='''
        Dash: A web application framework for Python.
    '''),

    dcc.Graph(
        id='example-graph',
        figure={
            'data': [
                {'x': ['United States of America', 'Canada', 'Australia', 'Chile'],'type': 'bar','y':  df_Country_count["Country"], 
                'type': 'line', 'name': 'SF'},
            ],
            'layout': {
                'title': 'Movimiento en los aereopuertos durante el covid'}
        }
        
    ),
    dcc.Graph(figure=fig3),
    
    dcc.Graph(figure=fig2),
    dcc.Graph(figure=fig4)
])

if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Jul/2021 05:03:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2021 05:03:26] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2021 05:03:26] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2021 05:03:26] "GET /_dash-component-suites/dash_core_components/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2021 05:03:26] "GET /_dash-component-suites/dash_core_components/async-plotlyjs.js HTTP/1.1" 200 -
